In [6]:
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import json
from pathlib import Path
import cv2
import mmcv
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [7]:
DATASET_BASE_PATH=Path('D:\SORD AI\Dataset')

In [8]:
# For coco dataset
# There are three necessary keys in the json file:

# images: contains a list of images with their information like file_name, height, width, and id.
# annotations: contains the list of instance annotations.
# categories: contains the list of categories names and their ID.


# {
#     "images": [image],
#     "annotations": [annotation],
#     "categories": [category]
# }


# image = {
#     "id": int,
#     "width": int,
#     "height": int,
#     "file_name": str,
# }
# annotation = {
#     "id": int,
#     "image_id": int,
#     "category_id": int,
#     "area": float,
#     "bbox": [x,y,width,height],
#     "iscrowd": 0 or 1,
# }
# categories = [{
#     "id": int,
#     "name": str,
#     "supercategory": str,
# }]

### Generating annotations

In [9]:
f= open(DATASET_BASE_PATH/'eval'/'objectclasses.json','r')
labels_info = json.load(f)

In [10]:
categories=[]
for cat in labels_info:
    categories.append({
        'name':cat['Name'],
        'id':cat['Id'],
    })

In [11]:
categories

[{'name': 'l_klt_4147', 'id': 1011},
 {'name': 'l_klt_6147', 'id': 1012},
 {'name': 'l_klt_8210', 'id': 1013},
 {'name': 'pallet', 'id': 1100},
 {'name': 'str', 'id': 2050},
 {'name': 'cabinet', 'id': 1040},
 {'name': 'locker', 'id': 1030},
 {'name': 'jack', 'id': 1120},
 {'name': 'dolly', 'id': 1110},
 {'name': 'spring_post', 'id': 1135},
 {'name': 'exit_sign', 'id': 4000},
 {'name': 'fire_extinguisher', 'id': 5010},
 {'name': 'stillage_open', 'id': 1003},
 {'name': 'stillage_close', 'id': 1002},
 {'name': 'cardboard_box', 'id': 1070},
 {'name': 'forklift', 'id': 2010},
 {'name': 'bicycle', 'id': 2000}]

In [14]:
def extract_image_annotations(img_name,folder):
    annotations=[]
    f= open(os.path.join(DATASET_BASE_PATH,'SORD_2022',folder,'labels','json',img_name.replace('jpg','json')))
    file_annotations=json.load(f)
    for annot_content in file_annotations:
        x=annot_content['Left']
        y=annot_content['Top']
        width=abs(annot_content['Right']-annot_content['Left'])
        height=abs(annot_content['Bottom']-annot_content['Top'])

        size=(width*height/H*W)
        size = (round(size*100,2))
        annotations.append({
            "id": img_id+'_'+str(annot_content['Id']),
            "image_id": img_id,
            "category_id": annot_content['ObjectClassId'],
            "bbox": [x, y, width, height],
            'area':width*height
        })
    return annotations

In [15]:
train_images=[]
train_annotations=[]

val_images=[]
val_annotations=[]

val_size = 0.30

for folder in os.listdir(DATASET_BASE_PATH/'SORD_2022'):
    print(folder)
    N=0
    img_id_prefix=''
    for x in (folder.split('_')[0:-1]):
        img_id_prefix=img_id_prefix+x
        img_id_prefix = img_id_prefix+'_'
    img_id_prefix = img_id_prefix[0:-1] 
    
#   Determining image size
    if '360p' in folder:
        H,W,C=(360,640,3)
    else:
        H,W,C=(720,1080,3)
#   Train Test Split
    folder_imgs=os.listdir(os.path.join(DATASET_BASE_PATH,'SORD_2022',folder,'images'))
    folder_imgs_train,folder_imgs_val=train_test_split(folder_imgs,test_size = val_size)
   
    N=0
    for i in tqdm(range(len(folder_imgs_train))):
        if N==10:
            break
        N=N+1
        img_name=folder_imgs_train[i]
        file_name=os.path.join(DATASET_BASE_PATH,'SORD_2022',folder,'images',img_name)
        img_id=img_id_prefix+'_'+img_name.split('.')[0]
        train_images.append({
            'id':img_id,
            'file_name':file_name,
            'height':H,
            'width':W
        })
        train_annotations=train_annotations+ extract_image_annotations(img_name,folder)
    N=0
    for i in tqdm(range(len(folder_imgs_val))):
        if N==10:
            break
        N=N+1
        img_name=folder_imgs_val[i]
        file_name=os.path.join(DATASET_BASE_PATH,'SORD_2022',folder,'images',img_name)
        img_id=img_id_prefix+'_'+img_name.split('.')[0]
        val_images.append({
            'id':img_id,
            'file_name':file_name,
            'height':H,
            'width':W
        })
        val_annotations=val_annotations+ extract_image_annotations(img_name,folder)

    
        


SORDI_2022_h4001_bicycle


  1%|█                                                                               | 10/750 [00:00<00:06, 118.53it/s]


SORDI_2022_h4002_cabinet


  1%|█                                                                               | 10/750 [00:00<00:03, 243.26it/s]


SORDI_2022_h4003_cardboard box


...

  0%|▏                                                                               | 10/5191 [00:00<01:39, 52.29it/s]


SORDI_2022_h4024_regensburg plant 360p


  0%|                                                                               | 10/17487 [00:00<10:28, 27.79it/s]


SORDI_2022_h4025_regensburg plant


  0%|                                                                               | 10/14265 [00:00<08:13, 28.86it/s]


In [16]:
print('Size of Train images',len(train_images))
print('Size of Train Annotations',len(train_annotations))

Size of Train images 240
Size of Train Annotations 4537


In [17]:
print('Size of Val images',len(val_images))
print('Size of Val Annotations',len(val_annotations))

Size of Val images 240
Size of Val Annotations 4204


In [18]:
train_annotation_name='train_sord_coco_annotations.json'
val_annotation_name='val_sord_coco_annotations.json'

In [19]:
coco_format_json = dict(
        images=train_images,
        annotations=train_annotations,
        categories=categories)
mmcv.dump(coco_format_json,train_annotation_name )

In [20]:
coco_format_json = dict(
        images=val_images,
        annotations=val_annotations,
        categories=categories)
mmcv.dump(coco_format_json,val_annotation_name )